In [1]:
import os
import sys
import argparse
import json
from datetime import datetime
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [2]:
load_dotenv()

True

In [3]:
def setup_client(api_key, endpoint, deployment_name):
    """
    Set up the Azure OpenAI client.

    Args:
        api_key (str): Azure OpenAI API key
        endpoint (str): Azure OpenAI endpoint URL
        deployment_name (str): Name of the deployed model

    Returns:
        AzureOpenAI client instance
    """
    client = AzureOpenAI(
        api_key=api_key,
        api_version="2024-02-15-preview",  # Make sure to use the correct API version for GPT-4o
        azure_endpoint=endpoint,
    )
    return client


def get_completion(
    client,
    deployment_name,
    system_message,
    user_prompt,
    temperature=0.7,
    max_tokens=1000,
):
    """
    Get a completion from the GPT-4o model.

    Args:
        client (AzureOpenAI): Azure OpenAI client
        deployment_name (str): Name of the deployed model
        system_message (str): System message to set context
        user_prompt (str): User prompt to get completion for
        temperature (float): Sampling temperature (0.0 to 1.0)
        max_tokens (int): Maximum tokens to generate

    Returns:
        str: Model's response
    """
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt},
            ],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [4]:
client = setup_client(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

In [5]:
quote1 = open("../data/quote_1.json", "r").read()
quote2 = open("../data/quote_2.json", "r").read()

user_prompt = f"""
Compare the following two quotes and provide an analysis of their similarities and differences.

Quote 1:

{quote1}

Quote 2:

{quote2}
"""

system_message = f"""
You are an insurance advisor who helps customers compare two given quotes and provide a summary of both while highlighting their key differences. Your goal is to assist the customer in making an informed decision based solely on the information provided in the quotes.

# Steps:

1. Review the details of both insurance quotes carefully.
2. Identify and summarize the key features of each quote, such as:
   - Coverage (e.g., type of coverage, limits, exclusions, etc.).
   - Total cost (e.g., premium, deductible, other charges, etc.).
   - Benefits (e.g., additional services, value-added features, etc.).
   - Limitations (e.g., restrictions, conditions, etc.).
3. Consider the major differences between the quotes, pointing out advantages and disadvantages for each quote.
4. Create key insights that are helpful in comparing the two quotes.
5. Avoid adding any outside information, personal opinions, or recommendations not supported by the provided data.
6. Format your output in a way that is clear, concise, and easy for customers to compare.

# Output Format:

Provide your response in the following format:

**Key features of each quote:**
[Table comparing the two quotes]

**Key Insights:**
- [Insights 1: Describe how Quote 1 and Quote 2 differ.]
- [Insights N: Additional differences, if necessary.]

**Final Note:**
Based on this comparison, the choice depends on the customer's [specific factors or preferences, e.g., budget, coverage type, additional benefits]. Both options align differently with various priorities.

# Notes:

- Be objective and fact-based. Avoid personal biases.
- Include all relevant differences, even if minor, as long as they might affect the customer’s decision.
- Encourage the customer to decide based on their priorities, such as budget, type of coverage, or additional benefits.
"""

In [6]:
response = get_completion(
    client=client,
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    system_message=system_message,
    user_prompt=user_prompt,
)
display(Markdown(response))

**Key features of each quote:**

| Feature                       | Quote 1                                              | Quote 2                                              |
|-------------------------------|-----------------------------------------------------|-----------------------------------------------------|
| **Quote Reference**           | 8048208602                                          | HAQ069514203                                        |
| **Total Excess**              | £300 (Voluntary: £250, Compulsory: £50)            | £350 (Voluntary: £250, Compulsory: £100)           |
| **Key Cover (Loss/Theft)**    | £1,000                                             | Up to contents sum insured                          |
| **Contents Cover Limit**      | £75,000                                            | Included (varies based on requested amount)        |
| **Single Item Limit**         | £2,000                                            | £1,500                                             |
| **Valuables Total Limit**     | 40% of contents sum insured                        | £5,000 - £30,000                                   |
| **Accidental Damage**         | Included                                           | Included                                           |
| **Legal Cover Cost**          | £24.85 extra                                      | £21.36 extra                                      |
| **Legal Expenses Claim Limit** | £100,000                                          | £50,000                                           |
| **Home Emergency Cover Cost** | £39.95 extra                                      | £54 extra                                         |
| **Home Emergency Claim Limit** | £500                                             | £1,000                                            |
| **Contents in Outbuildings**  | Up to contents sum insured                         | Not included                                       |
| **Contents in the Open**      | Not included                                       | Not included                                       |
| **Plants in Garden**          | £1,000                                             | Not included                                       |
| **Freezer Cover**             | Included                                           | Included                                           |
| **Personal Possessions Cover** | Not included                                       | Not included                                       |
| **Damage by Pets**            | Not included                                       | Not included                                       |

**Key Insights:**
- **Excess:** Quote 1 has a total excess of £300 compared to £350 in Quote 2, making Quote 1 slightly more favorable for claim situations.
- **Key Cover:** Quote 1 offers a fixed amount for loss/theft at £1,000, while Quote 2 covers up to the total contents sum insured, providing potentially higher coverage.
- **Contents Cover Limit:** Quote 1 clearly states a limit of £75,000, while Quote 2's limit is more variable, dependent on the requested amount, offering flexibility but less predictability.
- **Single Item Limit:** Quote 1 allows for a higher single item limit at £2,000 compared to £1,500 in Quote 2, which may be advantageous for high-value items.
- **Legal Cover Costs:** Quote 2 offers a lower cost for legal cover (£21.36) and has a lower claim limit (£50,000) compared to Quote 1 (£24.85 with a £100,000 limit).
- **Home Emergency Coverage:** Quote 2 provides a higher claim limit (£1,000) for home emergencies but at a higher cost (£54) than Quote 1 (£39.95 for £500).
- **Garden Protection:** Quote 1 provides £1,000 for plants in the garden, while Quote 2 does not include this coverage at all.

**Final Note:**
Based on this comparison, the choice depends on the customer's specific factors or preferences, such as budget considerations, coverage limits, and the importance of specific protections like garden coverage or higher single item limits. Both options align differently with various priorities, and customers should evaluate which features matter most to them.